In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import time

from IPython import display

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [2]:
noise_dim = 100
num_examples_to_generate = 16

seed = tf.random.normal([num_examples_to_generate, noise_dim])
BATCH_SIZE = 128 #8192

In [3]:
img_rows = 28
img_cols = 28
channels = 1

img_shape = (img_rows, img_cols, channels)

In [4]:
# Load the MNIST dataset
(X_train, _), (_, _) = mnist.load_data()

# Rescale [0, 255] grayscale pixel values to [-1, 1]
X_train = X_train / 127.5 - 1.0
X_train.shape
X_train = np.expand_dims(X_train, axis=3)
X_train = tf.data.Dataset.from_tensor_slices(X_train).batch(BATCH_SIZE)

In [5]:
def upscale_layer(layer, upscale_factor):
    height = layer.get_shape()[1]
    width = layer.get_shape()[2]
    size = (upscale_factor * height, upscale_factor * width)
    upscaled_layer = tf.image.resize_nearest_neighbor(layer, size)
    return upscaled_layer

In [6]:
def smoothly_merge_last_layer(list_of_layers, alpha):
    last_fully_trained_layer = list_of_layer[-2]
    last_layer_upscaled = upscale_layer(last_fully_trained_layer, 2)
    larger_native_layer = list_of_layers[-1]
    new_layer = (1-alpha) * upscaled_layer + larger_native_layer * alpha
    return new_layer

In [7]:
def build_generator(z_dim):
    model = tf.keras.Sequential()
    # Reshape input into 7x7x256 tensor via a fully connected layer
    model.add(layers.Dense(256 * 7 * 7, input_dim=z_dim))
    model.add(layers.Reshape((7, 7, 256)))

    # Transposed convolution layer, from 7x7x256 into 14x14x128 tensor
    model.add(layers.Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))

    # Batch normalization
    model.add(layers.BatchNormalization())

    # Leaky ReLU activation
    model.add(layers.LeakyReLU(alpha=0.01))

    # Transposed convolution layer, from 14x14x128 to 14x14x64 tensor
    model.add(layers.Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))

    # Batch normalization
    model.add(layers.BatchNormalization())

    # Leaky ReLU activation
    model.add(layers.LeakyReLU(alpha=0.01))

    # Transposed convolution layer, from 14x14x64 to 28x28x1 tensor
    model.add(layers.Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))

    # Output layer with tanh activation
    model.add(layers.Activation('tanh'))
    
    return model

In [8]:
def build_discriminator(img_shape):
    model = tf.keras.Sequential()
    # Convolutional layer, from 28x28x1 into 14x14x32 tensor
    model.add(layers.Input(shape=img_shape))
    model.add(
        layers.Conv2D(32,
               kernel_size=3,
               strides=2,
               #input_shape=img_shape,
               padding='same'))

    # Leaky ReLU activation
    model.add(layers.LeakyReLU(alpha=0.01))

    # Convolutional layer, from 14x14x32 into 7x7x64 tensor
    model.add(
        layers.Conv2D(64,
               kernel_size=3,
               strides=2,
               #input_shape=img_shape,
               padding='same'))

    # Batch normalization
    model.add(layers.BatchNormalization())

    # Leaky ReLU activation
    model.add(layers.LeakyReLU(alpha=0.01))

    # Convolutional layer, from 7x7x64 tensor into 3x3x128 tensor
    model.add(
        layers.Conv2D(128,
               kernel_size=3,
               strides=2,
               #input_shape=img_shape,
               padding='same'))

    # Batch normalization
    model.add(layers.BatchNormalization())

    # Leaky ReLU activation
    model.add(layers.LeakyReLU(alpha=0.01))

    # Output layer with sigmoid activation
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

In [9]:
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

In [10]:
discriminator = build_discriminator(img_shape)
generator = build_generator(noise_dim)

In [11]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)


In [12]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [13]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate_and_save_images(generator,
                                 epoch + 1,
                                 seed)

        # Save the model every 15 epochs
        #if (epoch + 1) % 15 == 0:
        #  checkpoint.save(file_prefix = checkpoint_prefix)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate_and_save_images(generator,
                           epochs,
                           seed)

In [14]:
def generate_and_save_images(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)

    fig = plt.figure(figsize=(4,4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    plt.show()


In [15]:
discriminator.layers.pop(0)#._layers[1].batch_input_shape = layers.Input(shape=(None, 28, 28, 3))

In [16]:
inputs = tf.keras.Input(shape=(3,))
x = tf.keras.layers.Dense(4, activation=tf.nn.relu)(inputs)
outputs = tf.keras.layers.Dense(5, activation=tf.nn.softmax)(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)


In [17]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 3)]               0         
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 16        
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 25        
Total params: 41
Trainable params: 41
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.outputs

[<KerasTensor: shape=(None, 5) dtype=float32 (created by layer 'dense_3')>]

In [19]:
newInput = tf.keras.Input(shape=(3,))
#newOutputs = discriminator(newInput)
newmodel = tf.keras.Model(inputs=newInput, outputs=model.outputs)

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 3), dtype=tf.float32, name='input_2'), name='input_2', description="created by layer 'input_2'") at layer "dense_2". The following previous layers were accessed without issue: []

In [ ]:
newInput = layers.Input(batch_shape=(0,299,299,3))    # let us say this new InputLayer
#newOutputs = discriminator(newInput)
discriminator = tf.keras.Model(newInput, discriminator)

In [ ]:
print(discriminator.input)
discriminator.summary()

In [ ]:
epochs = 20000

train(X_train, epochs)